In [3]:
# Librerías necesarias
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer

# Dataset ampliado con calaveritas adicionales
calaveritas_text = """
Juan andaba muy contento,
sin pensar en su tormento.
La Catrina lo miró,
y sin más se lo llevó.

La muerte buscaba almas
para llevarse al panteón,
vi mi calificación
y solito me fui yo.

Corriendo va la maestra,
Escapando de la muerte,
Al laboratorio entra,
Para poder esconderse.

Nadie te puede salvar,
Ni de mí te escaparás,
Hoy libero a los alumnos,
Te llevo al más allá.

Pero mis alumnos me aman,
No me dejarán marchar,
Son felices estudiando,
La química entienden ya.

Conocen símbolos químicos,
Pregunta y responderán,
Si aciertan el examen,
¿Libre me dejarás?

Pues hagamos una prueba,
¿Cuántos encontrarán?
En esta Calaverita
Bien escondidos están."

"Quien quiera gozar de veras
y divertirse un ratón,
venga con las calaveras
a gozar en el panteón.

Literatos distinguidos
en la hediondez encontré
en gusanos confundidos,
sin ellos saber por qué.
Y en gran tropel apiñados

Y en gran tropel apiñados
Los vendedores corrían
contentos y entusiasmados
por el negocio que hacían.

Cereros de sacristía
que roban la cera al rato,
que con mucha sangre fría
se echan el sufragio al plato.

La abuela en el altar La abuela estaba en su altar,
con flores, velas y pan,
cuando la Catrina pasó,
y a su casa la invitó,
¡pues el muerto se va a cenar!

"Ven abuela, no seas floja,
¡un mole quiero probar!
Con calabaza y tamales,
que el hambre me hace llorar,
y en la ofrenda se ha de gozar.

El profe estaba en la clase,
hablando de la revolución,
cuando la huesuda pasó,
y con un gran empujón,
¡lo mandó para su panteón!

Los vendedores corrían
contentos y entusiasmados
por el negocio que hacían.
Cereros de sacristía
que roban la cera al rato,
que con mucha sangre fría
se echan el sufragio al plato."

"De postre quiero calaveras,
de plato fuerte tamal,
en mi cocina macabra,
¡serás el chef principal,
de un banquete infernal!"

La diva estaba en su show,
cantando con gran pasión,
cuando la huesuda aplaudió,
y le dijo con emoción,
"¡Al panteón vamos de función!"

"Tu voz resonará en mi fiesta,
junto a mis calaveras fieles,
con tus canciones tan bellas,
celebraré los altares,
y tu fama por los umbrales."

El poeta escribía en calma,
versos de amor y de pena,
cuando la muerte le dijo,
"Soy yo tu última escena,
tu musa, tu última condena."

"Versos no me faltarán,
cuando juntos estemos,
serán rimas de ultratumba,
y poemas como premios,
en la eternidad de mis reinos."

Pintaba el artista su cuadro,
de colores y de ilusión,
cuando la muerte le dijo,
"Esta es tu última creación,
¡en mi galería habrá función!"

"Tus pinceles reposarán,
junto a mis sombras sin fin,
en un lienzo eterno estarán,
y de mi reino en el confín,
tu arte será un festín."

El músico tocaba alegre,
su guitarra y su tambor,
cuando la Catrina llegó,
y con ritmo de amor,
se lo llevó en un clamor.

"Toca, amigo, en mi fiesta,
que se alegren las tumbas,
en la orquesta de ultratumba,
serás la mejor nota,
de mi baile de calaveras juntas."

""" # Añade más calaveritas para mejorar los resultados

# Tokenizar el texto por palabras
tokenizer = Tokenizer()
tokenizer.fit_on_texts([calaveritas_text])

# Longitud del vocabulario
total_words = len(tokenizer.word_index) + 1

# Convertir texto a secuencias de palabras
input_sequences = []
for line in calaveritas_text.split("\n"):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# Padding de las secuencias
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre')

# Dividir en X e y
X, y = input_sequences[:,:-1], input_sequences[:,-1]
y = to_categorical(y, num_classes=total_words)

# Definir el modelo LSTM
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))

# Compilar el modelo
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entrenar el modelo
model.fit(X, y, epochs=200, verbose=1)

# Función para generar texto
def generate_calaverita(seed_text, next_words=50, diversity=0.5):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted_probs = model.predict(token_list, verbose=0)[0]

        # Reducir diversidad
        predicted_index = np.random.choice(range(total_words), p=predicted_probs)

        # Convertir índice a palabra
        output_word = tokenizer.index_word.get(predicted_index, "")
        seed_text += " " + output_word

    return seed_text

# Generar una calaverita
seed_text = "La muerte llegó"
print("Calaverita generada:\n", generate_calaverita(seed_text, next_words=30, diversity=0.2))


Epoch 1/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 44ms/step - accuracy: 0.0568 - loss: 5.5639
Epoch 2/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.0374 - loss: 5.3930
Epoch 3/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.0436 - loss: 5.2021
Epoch 4/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.0602 - loss: 5.0635
Epoch 5/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.0527 - loss: 5.0420
Epoch 6/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.0804 - loss: 4.9842
Epoch 7/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.0476 - loss: 4.9986
Epoch 8/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.0399 - loss: 4.9819
Epoch 9/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.0598 - loss: 4.9358
Epoch 10/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.0550 - loss: 4.9037
Epoch 11/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.0703 - loss: 4.7740
Epoch 12/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step